In [ ]:
from configparser import ConfigParser
import importlib
import threading

import mqtt
import jsonSerializer
import RTCmanager

In [ ]:
class Kiosk():
    def __init__(self, config):
        self.mqtt = mqtt.mqtt(config["mqtt"])
        self.RTCmanager=RTCmanager.RTCmanager()
        self.jsonSerializer=jsonSerializer.jsonGen()
           
    def run(self):
        thread = threading.Thread(target=self.mqtt.run)
        thread.daemon=True
        thread.start()
        msgQueue = [self.RTCmanager.spo2.msgQueue,self.RTCmanager.raycomRPD7000.msgQueue]
        while True:
            if not self.mqtt.message.empty():
                cmd = self.mqtt.message.get_nowait()
                self.RTCmanager.analyze(cmd)
            if any(msgQueue):
                for msg in msgQueue:
                    try:
                        message = msg.get_nowait()
                        self.jsonSerializer.jsonManager(message)
                        while not self.jsonSerializer.msgQueue.empty():
                            if eval(cmd.payload)['cmd'] == "status":
                                
                                self.mqtt.publish("remoteCtrlReply",self.jsonSerializer.msgQueue.get_nowait())
                            else:
                                self.mqtt.publish("vitalSign",self.jsonSerializer.msgQueue.get_nowait())
                    except Exception as e:
                        pass
        


In [ ]:
configFile={"hardware":"config/banbangkhae.ini",
            "commProtocol":"config/emetWorksCommProtocol.ini"
           }
config=ConfigParser()
config.read(configFile["commProtocol"])
configDevice= ConfigParser()
configDevice.read(configFile["hardware"])


kiosk=Kiosk(config)

kiosk.run()

In [ ]:
spo2BlackBoxThread = threading.Thread(target=kiosk.spo2BlackBox.main, name="spo2BlackBox")
spo2BlackBoxThread.daemon=True
spo2BlackBoxThread.start()

In [ ]:
kiosk.spo2BlackBox.action="status"

In [ ]:
kiosk.spo2BlackBox.action="start"

In [ ]:
obj=kiosk.spo2BlackBox.msgQueue.get_nowait()

In [ ]:
kiosk.spo2BlackBox.msgQueue.empty()

In [ ]:
kiosk.raycomRPD7000.action = "connect"

In [ ]:
kiosk.raycomRPD7000.action = "start"

In [ ]:
kiosk.raycomRPD7000.action = "stop"

In [ ]:
kiosk.raycomRPD7000.action = "status"

In [ ]:
obj=kiosk.raycomRPD7000.msgQueue.get()

In [ ]:
obj.__dict__

In [ ]:
mqttThread=threading.Thread(target=kiosk.mqtt)
mqttThread.daemon=True
mqttThread.start()

In [ ]:
kiosk.mqttObj.publish("vitalSign", 1)

In [ ]:
kiosk.mqttObj.message.get_nowait()

In [ ]:
import json

In [ ]:
json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])